In [2]:
def filterData(row, dados_csv, data_hora_ini, data_hora_fim, insert_queimada, insert_queimada_comp, isRange):
    data_hora_csv_str = re.search(r'[0-9]{4}-[0-9]{2}-[0-9]{2} [0-9]{2}:[0-9]{2}:[0-9]{2}', row['data_hora_gmt'])
    data_hora_csv = datetime.strptime(data_hora_csv_str[0], "%Y-%m-%d %H:%M:%S")
            
    if(data_hora_ini <= data_hora_csv <= data_hora_fim):
                
        foreign_key = insertTable(row)                                        # database
        createInsert(row, insert_queimada, insert_queimada_comp, foreign_key) # script
                    
        dados_csv = np.append(dados_csv, row)

        isRange = True
    if (data_hora_csv > data_hora_fim):
        return 0

In [3]:
def stateToAcronym(estado):
  match(estado):
      case 'ACRE':
        estado = 'AC'

      case 'ALAGOAS':
        estado = 'AL'

      case 'AMAPÁ':
        estado = 'AP'

      case 'AMAZONAS':
        estado = 'AM'

      case 'BAHIA':
        estado = 'BA'

      case 'CEARÁ':
        estado = 'CE'

      case 'ESPÍRITO SANTO':
        estado = 'ES'

      case 'GOIÁS':
        estado = 'GO'

      case 'MARANHÃO':
        estado = 'MA'

      case 'MATO GROSSO':
        estado = 'MT'

      case 'MATO GROSSO DO SUL':
        estado = 'MS'

      case 'MINAS GERAIS':
        estado = 'MG'

      case 'PARÁ':
        estado = 'PA'

      case 'PARAÍBA':
        estado = 'PB'

      case 'PARANÁ':
        estado = 'PR'

      case 'PERNAMBUCO':
        estado = 'PE'

      case 'PIAUÍ':
        estado = 'PI'

      case 'RIO DE JANEIRO':
        estado = 'RJ'

      case 'RIO GRANDE DO NORTE':
        estado = 'RN'

      case 'RIO GRANDE DO SUL':
        estado = 'RS'

      case 'RONDÔNIA':
        estado = 'RO'

      case 'RORAIMA':
        estado = 'RR'

      case 'SANTA CATARINA':
        estado = 'SC'

      case 'SÃO PAULO':
        estado = 'SP'

      case 'SERGIPE':
        estado = 'SE'

      case 'TOCANTINS':
        estado = 'TO'

      case 'DISTRITO FEDERAL':
        estado = 'DF'
  return estado

In [4]:
def createInsert(row, insert_queimada, insert_queimada_comp, foreign_key):
    latitude  = row['lat'].strip()
    longitude = row['lon'].strip()
    municipio = ftfy.fix_text(row['municipio'].replace("'","''"))
    
    estado = stateToAcronym(ftfy.fix_text(row['estado']))
        
    insert_queimada.append(f"INSERT INTO QUEIMADA (DataHora, Latitude, Longitude, Estado, Municipio, Satelite) VALUES ('{row['data_hora_gmt']}', '{latitude}', '{longitude}', '{estado}', '{municipio}', '{row['satelite']}');\n")
    insert_queimada_comp.append(f"INSERT INTO QUEIMADA_COMPLEMENTO (COD_QUEIM, CAMPO, VALOR) VALUES ('{foreign_key}', 'Bioma', '{row['bioma']}');\n")
    insert_queimada_comp.append(f"INSERT INTO QUEIMADA_COMPLEMENTO (COD_QUEIM, CAMPO, VALOR) VALUES ('{foreign_key}', 'Diasemchuva', '{row['numero_dias_sem_chuva']}');\n")
    insert_queimada_comp.append(f"INSERT INTO QUEIMADA_COMPLEMENTO (COD_QUEIM, CAMPO, VALOR) VALUES ('{foreign_key}', 'Precipitacao', '{row['precipitacao']}');\n")
    insert_queimada_comp.append(f"INSERT INTO QUEIMADA_COMPLEMENTO (COD_QUEIM, CAMPO, VALOR) VALUES ('{foreign_key}', 'Riscofogo', '{row['risco_fogo']}');\n")    
    
    

In [9]:
import pyodbc

def connection_database():
    server = "DJ10W9D3\SQLEXPRESS"
    database = "bd_sinape_alupar"
    username = "sa"
    password = "DAS@sinap"
    
    connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};sslverify=0'
    return connection_string

In [6]:
def insertTable(row):
    id = 0
    try:
        connection = pyodbc.connect(connection_database())
        cursor = connection.cursor()
        
        cursor.execute('SELECT MAX(COD_QUEIM) FROM QUEIMADA')
        result_set = cursor.fetchall()
    
        if result_set and result_set[0][0] is not None:
            id = result_set[0][0] + 1
        else:
            id = 1

    
        latitude  = row['lat'].strip()
        longitude = row['lon'].strip()
        municipio = ftfy.fix_text(row['municipio'].replace("'","''"))
        estado    = stateToAcronym(ftfy.fix_text(row['estado']))
        satelite  = ftfy.fix_text(row['satelite'])
        
        bioma        = ftfy.fix_text(row['bioma'])
        chuva        = ftfy.fix_text(row['numero_dias_sem_chuva'])
        precipitacao = ftfy.fix_text(row['precipitacao'])
        fogo         = ftfy.fix_text(row['risco_fogo'])
        
        
        sql_insert_queimada = "INSERT INTO QUEIMADA (DataHora, Latitude, Longitude, Estado, Municipio, Satelite) VALUES (?, ?, ?, ?, ?, ?);"
        values_queimada = (row['data_hora_gmt'], latitude, longitude, estado, municipio, satelite)
        
        sql_insert_queimada_complemento_bioma = "INSERT INTO QUEIMADA_COMPLEMENTO (COD_QUEIM, CAMPO, VALOR) VALUES (?, ?, ?)"
        values_complemento_bioma = (id, 'Bioma', bioma)
        
        sql_insert_queimada_complemento_chuva = "INSERT INTO QUEIMADA_COMPLEMENTO (COD_QUEIM, CAMPO, VALOR) VALUES (?, ?, ?)"
        values_complemento_chuva = (id, 'Diasemchuva', chuva)
        
        sql_insert_queimada_complemento_precipitacao = "INSERT INTO QUEIMADA_COMPLEMENTO (COD_QUEIM, CAMPO, VALOR) VALUES (?, ?, ?)"
        values_complemento_precipitacao = (id, 'Precipitacao', precipitacao)
        
        sql_insert_queimada_complemento_fogo = "INSERT INTO QUEIMADA_COMPLEMENTO (COD_QUEIM, CAMPO, VALOR) VALUES (?, ?, ?)"
        values_complemento_fogo = (id, 'Riscofogo', fogo)
        
        cursor.execute(sql_insert_queimada, values_queimada)
        cursor.execute(sql_insert_queimada_complemento_bioma, values_complemento_bioma)
        cursor.execute(sql_insert_queimada_complemento_chuva, values_complemento_chuva)
        cursor.execute(sql_insert_queimada_complemento_precipitacao, values_complemento_precipitacao)
        cursor.execute(sql_insert_queimada_complemento_fogo, values_complemento_fogo)
    
        connection.commit()
             
    except Exception as e:
        print(f'Error in connection: {e}')
        
    finally:
        if 'cursor' in locals():
            cursor.close()
            
        if 'connection' in locals():
            connection.close()
            
    return id

In [ ]:
import re
import csv
import json
import ftfy
import codecs
import numpy as np
from datetime import datetime

import requests
import csv
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

ano = "2023"
mes = "10"

url = f'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/mensal/Brasil/focos_mensal_br_{ano}{mes}.csv'

arquivo_csv  = 'arquivo.csv'
arquivo_text = 'insert.txt'
arquivo_json = 'arquivo.json'
arquivo_text_comp = 'insert_comp.txt'

dados_csv_np         = np.array([])

insert_queimada      = []
insert_queimada_comp = []

isRange = False
foreign_key = 0

data_hora_ini_str = re.search(r'[0-9]{4}-[0-9]{2}-[0-9]{2} [0-9]{2}:[0-9]{2}:[0-9]{2}', '2023-10-21 00:00:00')
data_hora_fim_str = re.search(r'[0-9]{4}-[0-9]{2}-[0-9]{2} [0-9]{2}:[0-9]{2}:[0-9]{2}', '2023-10-23 23:59:59')

data_hora_ini = datetime.strptime(data_hora_ini_str[0], "%Y-%m-%d %H:%M:%S")
data_hora_fim = datetime.strptime(data_hora_fim_str[0], "%Y-%m-%d %H:%M:%S")

rangeTeste = 0

with requests.get(url, stream=True, verify=False) as response:
    if response.status_code == 200:
        dados_csv = response.iter_lines(decode_unicode=True)
        
        csv_reader = csv.DictReader(dados_csv)        
        for row in csv_reader:
            
            data_hora_csv_str = re.search(r'[0-9]{4}-[0-9]{2}-[0-9]{2} [0-9]{2}:[0-9]{2}:[0-9]{2}', row['data_hora_gmt'])
            data_hora_csv = datetime.strptime(data_hora_csv_str[0], "%Y-%m-%d %H:%M:%S")
            
            if(data_hora_ini <= data_hora_csv <= data_hora_fim):
                
                foreign_key = insertTable(row)                                        # database
                createInsert(row, insert_queimada, insert_queimada_comp, foreign_key) # script
                dados_csv_np = np.append(dados_csv_np, row)                           # json

                isRange = True
            if (data_hora_csv > data_hora_fim):
                break
                
           # breakpoint = filterData(row, dados_csv, data_hora_ini, data_hora_fim, insert_queimada, insert_queimada_comp, isRange)
           # if(breakpoint == 0):
           #     break

        
if isRange:
    novo_dados = dados_csv_np[1:].tolist()
    novo_dados_csv = json.dumps(novo_dados, ensure_ascii=False, indent=4)
    novo_dados_json = ftfy.fix_text(novo_dados_csv)
    with codecs.open(arquivo_json, 'w', encoding='utf-8') as arquivo:
        arquivo.write(novo_dados_json)
    
    with codecs.open(arquivo_text, 'w', encoding='utf-8') as arquivo:
        for linha in insert_queimada:
            arquivo.write(f"{ftfy.fix_text(linha)}")
            
    with codecs.open(arquivo_text_comp, 'w', encoding='utf-8') as arquivo:
        for linha in insert_queimada_comp:
            arquivo.write(f"{ftfy.fix_text(linha)}")
    
    print(f'Conversão concluída. Os dados JSON foram salvos em {arquivo_json}')
    print(f'Conversão concluída. Os dados TXT foram salvos em {arquivo_text}')
    print(f'Conversão concluída. Os dados TXT foram salvos em {arquivo_text_comp}')